# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

# reference
https://stackoverflow.com/questions/27298178/concatenate-strings-from-several-rows-using-pandas-groupby/45925961
https://stackoverflow.com/questions/51102205/how-to-know-the-labels-assigned-by-astypecategory-cat-codes
https://stackoverflow.com/questions/45312377/how-to-one-hot-encode-from-a-pandas-column-containing-a-list
https://stackoverflow.com/questions/50522146/pandas-column-dict-split-to-new-column-and-rows
https://stackoverflow.com/questions/56421683/how-to-apply-a-function-for-each-row-of-a-dataframe-which-checks-keys-of-embedd
https://stackoverflow.com/questions/41455967/convert-datetime-string-to-new-columns-of-day-month-year-in-pandas-data-frame/41456152

https://stackoverflow.com/questions/43133605/convert-integer-yyyymmdd-to-date-format-mm-dd-yyyy-in-python
https://stackoverflow.com/questions/17134716/convert-dataframe-column-type-from-string-to-datetime-dd-mm-yyyy-format

https://stackoverflow.com/questions/49742938/checking-isdigit-on-python


#  : https://towardsdatascience.com/flattening-json-objects-in-python-f5343c794b10

In [1]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import matplotlib
#% matplotlib inline|

# read in the json files
portfolio = pd.read_json('portfolio.json', orient='records', lines=True)
profile = pd.read_json('profile.json', orient='records', lines=True)
transcript = pd.read_json('transcript.json', orient='records', lines=True)

# Part 1 :  Initial Dataset Statistics and basic EDA

## summary of portfolio statistics
- profile dataframe has age, gender catgorical features which we will need to encode
- only income and gender columns have more missing values - 2175
- become member on need to be changed to todal time they are member
- age max value 118 - looks like outlie

## summary of transcript statistics
- profile dataframe has age, gender catgorical features which we will need to encode
- only income and gender columns have more missing values - 2175
- become member on need to be changed to todal time they are member
- age max value 118 - looks like outlie

## summary of profile statistics
- profile dataframe has age, gender catgorical features which we will need to encode
- only income and gender columns have more missing values - 2175
- become member on need to be changed to todal time they are member
- age max value 118 - looks like outlier



In [2]:
# Check size of the data set
print('portfolio  shape is {}:' .format(portfolio.shape)) 
print('profile  shape is {}:' .format(profile.shape))   
print('transcript  shape is {}: ' .format(transcript.shape)) 



portfolio  shape is (10, 6):
profile  shape is (17000, 5):
transcript  shape is (306534, 4): 


In [3]:
# Check size of the basic statistics
print('portfolio statistics  {}:' .format(portfolio.describe())) 
print('portfolio data   {}:' .format(portfolio.info())) 

portfolio statistics         difficulty   duration     reward
count   10.000000  10.000000  10.000000
mean     7.700000   6.500000   4.200000
std      5.831905   2.321398   3.583915
min      0.000000   3.000000   0.000000
25%      5.000000   5.000000   2.000000
50%      8.500000   7.000000   4.000000
75%     10.000000   7.000000   5.000000
max     20.000000  10.000000  10.000000:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
channels      10 non-null object
difficulty    10 non-null int64
duration      10 non-null int64
id            10 non-null object
offer_type    10 non-null object
reward        10 non-null int64
dtypes: int64(3), object(3)
memory usage: 560.0+ bytes
portfolio data   None:


In [4]:
print('profile statistics  {}:' .format(profile.describe())) 
print('profile data   {}:' .format(profile.info()))  


profile statistics                  age  became_member_on         income
count  17000.000000      1.700000e+04   14825.000000
mean      62.531412      2.016703e+07   65404.991568
std       26.738580      1.167750e+04   21598.299410
min       18.000000      2.013073e+07   30000.000000
25%       45.000000      2.016053e+07   49000.000000
50%       58.000000      2.017080e+07   64000.000000
75%       73.000000      2.017123e+07   80000.000000
max      118.000000      2.018073e+07  120000.000000:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
age                 17000 non-null int64
became_member_on    17000 non-null int64
gender              14825 non-null object
id                  17000 non-null object
income              14825 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.1+ KB
profile data   None:


Outlier in transcript 'age' column

In [5]:
print('transcript statistics  {}:' .format(profile.describe())) 
print('transcript data   {}:' .format(profile.info()))  

transcript statistics                  age  became_member_on         income
count  17000.000000      1.700000e+04   14825.000000
mean      62.531412      2.016703e+07   65404.991568
std       26.738580      1.167750e+04   21598.299410
min       18.000000      2.013073e+07   30000.000000
25%       45.000000      2.016053e+07   49000.000000
50%       58.000000      2.017080e+07   64000.000000
75%       73.000000      2.017123e+07   80000.000000
max      118.000000      2.018073e+07  120000.000000:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
age                 17000 non-null int64
became_member_on    17000 non-null int64
gender              14825 non-null object
id                  17000 non-null object
income              14825 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.1+ KB
transcript data   None:


Only need to deal with null value in profile - gender and income. 


In [6]:
# Check size of the statistics
print('portfolio null count {}:' .format(portfolio.isnull().sum())) 
print('profile null count{}:' .format(profile.isnull().sum()))   
print('transcript null count{}: ' .format(transcript.isnull().sum())) 

portfolio null count channels      0
difficulty    0
duration      0
id            0
offer_type    0
reward        0
dtype: int64:
profile null countage                    0
became_member_on       0
gender              2175
id                     0
income              2175
dtype: int64:
transcript null countevent     0
person    0
time      0
value     0
dtype: int64: 


In [7]:
portfolio.head()

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5


In [8]:
# cleaning function portfolio
# credit -  https://stackoverflow.com/questions/45312377/how-to-one-hot-encode-from-a-pandas-column-containing-a-list
# credit - https://stackoverflow.com/questions/29034928/pandas-convert-a-column-of-list-to-dummies

def portfolio_clean(portfolio):

    """ 
    Clean, select and transform a DataFrame containing information about offer type and duration
    INPUT:
        portfolio: DataFrame containing offer ids and meta data about 
                   each offer (duration, type, etc.)
    OUTPUT:
        portfolio_df: clean transformed portfolio DataFrame
    """  
    
    offer_id_col = {'ae264e3637204a6fb9bb56bc8210ddfd':'bogo1',
       '4d5c57ea9a6940dd891ad53e9dbe8da0':'bogo2',
       '3f207df678b143eea3cee63160fa8bed':'informational1',
       '9b98b8c7a33c4b65b9aebfe6a799e6d9': 'bogo3',
       '0b1e1539f2cc45b7b9fa7c272da2e1d7':'discount1',
       '2298d6c36e964ae4a3e7e9706d1fb8c2':'discount2',
       'fafdcd668e3743c1bb461111dcafc2a4':'discount3',
       '5a8bc65990b245e5a138643cd4eb9837':'informational2',
       'f19421c1d4aa40978ebb69ca19b0e20d':'bogo4',
       '2906b810c7d4411798c6938adc9daaa5':'discount4'}

    
    portfolio_df=portfolio.assign(**pd.get_dummies(portfolio.channels.apply(lambda x:pd.Series(x)).stack().reset_index(level=1,drop=True)).sum(level=0))

    portfolio_df['id'] = [offer_id_col[item] for item in portfolio_df['id']] 
    portfolio_df.rename(columns={'id':'offer_id'}, inplace =True)
    portfolio_df.drop(['offer_type', 'channels'], axis=1,inplace=True)
    
    return portfolio_df



In [9]:
portfolio_df=portfolio_clean(portfolio)

In [11]:
portfolio_df.head()

,difficulty,duration,offer_id,reward,email,mobile,social,web
0,10,7,bogo1,10,1,1,1,0
1,10,5,bogo2,10,1,1,1,1
2,0,4,informational1,0,1,1,0,1
3,5,7,bogo3,5,1,1,0,1
4,20,10,discount1,5,1,0,0,1


In [19]:
col_portfolio= [portfolio_df.columns.tolist()]
col_portfolio

[['difficulty',
  'duration',
  'offer_id',
  'reward',
  'email',
  'mobile',
  'social',
  'web']]

In [ ]:

col_portfolio= ['difficulty', 'duration','reward', 'email', 'mobile', 'offer_id', 'social', 'web']

for column_name in col_portfolio:
    ax=plt.subplots(figsize=(6,3))
    ax=portfolio_df[column_name].value_counts().plot(kind='bar')
    title="Histogram of " + column_name
    plt.title(title, fontsize=12)
    plt.show()

In [33]:
# cleaning function profile
# credit -  https://stackoverflow.com/questions/43133605/convert-integer-yyyymmdd-to-date-format-mm-dd-yyyy-in-python


def profile_clean(profile):

    """ 
    Clean, select and transform a DataFrame containing a demographic data for each customer
    INPUT:
        profile: DataFrame containing offer ids and meta data about 
                   each offer (duration, type, etc.)
    OUTPUT:
        profile_df: clean and transformed profile DataFrame
    """  
    

    profile['date_time']  = profile[['became_member_on']].applymap(str).applymap(lambda s: "{}/{}/{}"
                                                                                 .format( s[0:4],s[4:6],s[6:]))
                            
    # only gender and income column showed nan values. We will clean the data
    profile= profile.dropna(axis=0, subset=['gender'])
    
    # keep only year unless want to calculate membership period then we may need month and date 
    profile['year'] = pd.DatetimeIndex(profile['date_time']).year   

    profile.rename(columns={'id':'customer_id'}, inplace =True)
    profile.drop(['date_time', 'became_member_on'], axis=1,inplace=True)

    return profile





In [34]:
profile_df=profile_clean(profile)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [35]:
profile_df.head()

,age,gender,customer_id,income,year
1,55,F,0610b486422d4921ae7d2bf64640c50b,112000.0,2017
3,75,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0,2017
5,68,M,e2127556f4f64592b11af22de27a7932,70000.0,2018
8,65,M,389bc3fa690240e798340f5a15918d5c,53000.0,2018
12,58,M,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,2017


In [ ]:
# cleaning function transcript
# credit -  https://stackoverflow.com/questions/43133605/convert-integer-yyyymmdd-to-date-format-mm-dd-yyyy-in-python


def transcript_clean(transcript):

    """ 
    Clean, select and transform a DataFrame containing the records for transactions,
    offer types : received, viewed, completed
    INPUT:
        transcript: DataFrame containing records of transaction, customer id, time and value
                   each offer (duration, type, etc.)
    OUTPUT:
        transcript_df: clean and transformed transcript DataFrame
    """  
    # value column contains disctionary with amount, reward and offer id 
    transcript = pd.concat([transcript, transcript['value'].apply(pd.Series)], axis = 1).drop('value', axis = 1)
    
    profile.rename(columns={'id':'customer_id'}, inplace =True)
    profile.drop(['date_time', 'became_member_on'], axis=1,inplace=True)

    return transcript

In [ ]:
col_profile= [profile_df.columns.tolist()]

for column_name in col_portfolio:
    ax=plt.subplots(figsize=(6,3))
    ax=portfolio_df[column_name].value_counts().plot(kind='bar')
    title="Histogram of " + column_name
    plt.title(title, fontsize=12)
    plt.show()

In [ ]:
for column_name in col_profile:
    ax1=plt.subplots(figsize=(6,3))
    # get data by column_name and display a histogram
    ax1 = plt.hist(profile_df[column_name], bins=n_bins)
    title="Histogram of " + column_name
    plt.title(title, fontsize=12)
    plt.show()


In [ ]:
transcript

In [ ]:
  df = pd.merge(transcript, profile, on='persone_id')

In [ ]:

    df = pd.merge(transcript, profile, on='persone_id')
    df = pd.merge(df, portfolio, on='offer_id', how='left')
    return df